<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=099ff6b5555350551ea3eb155297738c6b0b1ea6c5e343e924aba185ff0b4be0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 14:09:40
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 81.55 K (0.58%)
Current PnL: -20.41 L (-13.6%)
CY Booked + Current PnL: -8.51 L (-5.67%)
-------------------
Total profit:  2.21 L
Total loss:  -22.62 L
-------------------
Total Booked + Current PnL: 18.47 L (15.0%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.44%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.58 L (62.08%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-16.01,71.0,X-LC,4.02,224255.0,19755.0,3543.0,3.76,9.66,1.58,11.39,10.0,5.58,1.60,19.48,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-8.41,74.0,X-LC,2.89,229180.0,18814.0,9969.0,3.69,8.94,4.35,13.68,37.0,1.89,1.64,26.94,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,3.26,63.0,X-MC,1.58,216555.0,24813.0,12950.0,1.20,12.94,5.98,19.69,99.0,1.92,1.55,20.62,XY25,NTT,AC
76,TTKPRESTIG,770.00,95.09,46.0,M-SC,1.71,84076.0,-16701.0,16798.0,-0.06,-16.57,19.98,0.09,245.0,-0.99,0.60,10.46,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.65,148725.0,12592.0,18531.0,0.44,9.25,12.46,22.86,91.0,0.68,1.06,23.03,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SURYODAY,240.0,66.67,62.0,H-SC,11.17,152235.0,-26836.0,92559.0,5.63,-14.99,60.80,36.71,135.0,-0.29,1.09,51.05,XR,NTT,BANKS
11,BANKINDIA,190.0,-22.98,66.0,H-MC,12.59,199832.0,20024.0,92382.0,5.54,11.14,46.23,62.52,88.0,0.22,1.43,49.02,XR,NTT,BANKS
77,UJJIVANSFB,60.0,119.37,69.0,H-SC,12.98,138429.0,-4050.0,23574.0,5.23,-2.84,17.03,13.70,163.0,-0.17,0.99,63.38,OX40N,NTT,BANKS
10,BANDHANBNK,400.0,-9.90,56.0,H-SC,2.92,224758.0,-53801.0,311245.0,4.19,-19.31,138.48,92.42,151.0,-0.17,1.61,29.89,XY24,NTT,BANKS
78,UNIONBANK,163.0,-10.08,60.0,M-LC,8.81,161693.0,20853.0,23640.0,4.04,14.81,14.62,31.59,66.0,0.88,1.16,45.22,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-26.08,38.0,H-SC,18.75,197936.0,-72825.0,402780.0,-4.11,-26.90,203.49,121.86,133.0,-0.18,1.41,54.84,AR,ATH,IT
31,HINDZINC,730.22,29.80,53.0,M-LC,10.12,212084.0,7008.0,105554.0,-2.54,3.42,49.77,54.89,52.0,0.07,1.52,28.34,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101258.0,9760.0,68997.0,-2.48,10.67,68.14,86.07,223.0,0.14,0.72,47.04,XR,NTT,DURABLES
67,STARHEALTH,761.00,21.02,66.0,H-SC,9.77,269416.0,-11707.0,144569.0,-1.73,-4.16,53.66,47.26,144.0,-0.08,1.92,44.35,XY24,NTT,INSURANCE
24,EASEMYTRIP,26.40,-19.28,39.0,M-SC,25.93,77771.0,-99044.0,179838.0,-1.36,-56.02,231.24,45.70,197.0,-0.55,0.56,1.53,XY24,NTT,TRAVEL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,47.0,M-MC,10.85,223998.0,-964.0,168626.0,1.09,-0.43,75.28,74.53,192.0,-0.01,1.60,31.69,XY24,BTT,STEEL
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124337.0,-960.0,59893.0,3.12,-0.77,48.17,47.03,141.0,-0.02,0.89,55.68,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,119.37,69.0,H-SC,12.98,138429.0,-4050.0,23574.0,5.23,-2.84,17.03,13.70,163.0,-0.17,0.99,63.38,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,60.0,H-SC,1.91,227259.0,-42408.0,78745.0,-0.30,-15.73,34.65,13.47,138.0,-0.54,1.62,15.55,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.09,46.0,M-SC,1.71,84076.0,-16701.0,16798.0,-0.06,-16.57,19.98,0.09,245.0,-0.99,0.60,10.46,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.11,40.0,H-SC,2.48,138426.0,-37477.0,79498.0,0.28,-21.31,57.43,23.89,149.0,-0.47,0.99,19.55,OX40N,NTT,CERAMICS
65,SIS,528.0,2049.74,50.0,H-SC,3.82,86751.0,-24281.0,47887.0,1.22,-21.87,55.20,21.26,156.0,-0.51,0.62,17.21,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,51.0,M-LC,6.04,206800.0,3630.0,38506.0,0.28,1.79,18.62,20.74,55.0,0.09,1.48,7.43,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,53.0,M-LC,10.12,212084.0,7008.0,105554.0,-2.54,3.42,49.77,54.89,52.0,0.07,1.52,28.34,X5K,ATH,METALS
78,UNIONBANK,163.00,-10.08,60.0,M-LC,8.81,161693.0,20853.0,23640.0,4.04,14.81,14.62,31.59,66.0,0.88,1.16,45.22,XY24,NTT,BANKS
11,BANKINDIA,190.00,-22.98,66.0,H-MC,12.59,199832.0,20024.0,92382.0,5.54,11.14,46.23,62.52,88.0,0.22,1.43,49.02,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101258.0,9760.0,68997.0,-2.48,10.67,68.14,86.07,223.0,0.14,0.72,47.04,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-22.98,66.0,H-MC,12.59,199832.0,20024.0,92382.0,5.54,11.14,46.23,62.52,88.0,0.22,1.43,49.02,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101258.0,9760.0,68997.0,-2.48,10.67,68.14,86.07,223.0,0.14,0.72,47.04,XR,NTT,DURABLES
31,HINDZINC,730.22,29.80,53.0,M-LC,10.12,212084.0,7008.0,105554.0,-2.54,3.42,49.77,54.89,52.0,0.07,1.52,28.34,X5K,ATH,METALS
36,INDIAMART,4810.62,-55.20,41.0,H-SC,7.76,124873.0,1537.0,130093.0,0.51,1.25,104.18,106.72,119.0,0.01,0.89,24.92,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149395.0,-1550.0,111000.0,0.02,-1.03,74.30,72.51,53.0,-0.01,1.07,4.93,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,144891.0,-131632.0,241707.0,0.64,-47.60,166.82,39.81,54.0,-0.54,1.04,2.12,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,37.0,X-SC,38.75,52056.0,-12950.0,163877.0,1.54,-19.92,314.81,232.18,198.0,-0.08,0.37,1.54,XY24,NTT,MISC
9,BAJAJHFL,181.50,-15.11,41.0,X-MC,5.47,178908.0,-21598.0,118759.0,-0.31,-10.77,66.38,48.45,90.0,-0.18,1.28,3.17,X40N,ATH,FINANCE
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182730.0,-55221.0,207874.0,-0.29,-23.21,113.76,64.15,174.0,-0.27,1.31,2.50,XY24,BTT,CEMENT
28,HAVELLS,2069.16,-6.75,42.0,X-MC,2.38,231208.0,-16523.0,91581.0,-0.63,-6.67,39.61,30.30,92.0,-0.18,1.65,5.74,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,44.0,H-LC,0.36,155680.0,-30415.0,77902.0,-0.24,-16.34,50.04,25.51,15.0,-0.39,1.11,14.72,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,3.26,63.0,X-MC,1.58,216555.0,24813.0,12950.0,1.20,12.94,5.98,19.69,99.0,1.92,1.55,20.62,XY25,NTT,AC
28,HAVELLS,2069.16,-6.75,42.0,X-MC,2.38,231208.0,-16523.0,91581.0,-0.63,-6.67,39.61,30.30,92.0,-0.18,1.65,5.74,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201835.0,1697.0,19195.0,0.15,0.85,9.51,10.44,4.0,0.09,1.44,6.46,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.65,148725.0,12592.0,18531.0,0.44,9.25,12.46,22.86,91.0,0.68,1.06,23.03,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,37.0,X-SC,38.75,52056.0,-12950.0,163877.0,1.54,-19.92,314.81,232.18,198.0,-0.08,0.37,1.54,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,144891.0,-131632.0,241707.0,0.64,-47.60,166.82,39.81,54.0,-0.54,1.04,2.12,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182730.0,-55221.0,207874.0,-0.29,-23.21,113.76,64.15,174.0,-0.27,1.31,2.50,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-15.11,41.0,X-MC,5.47,178908.0,-21598.0,118759.0,-0.31,-10.77,66.38,48.45,90.0,-0.18,1.28,3.17,X40N,ATH,FINANCE
73,TCS,4389.96,-29.01,51.0,X-LC,12.73,283316.0,-62640.0,129334.0,1.75,-18.11,45.65,19.28,1.0,-0.48,2.02,4.74,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,37.0,X-SC,38.75,52056.0,-12950.0,163877.0,1.54,-19.92,314.81,232.18,198.0,-0.08,0.37,1.54,XY24,NTT,MISC
55,RELAXO,1176.00,-39.97,42.0,X-SC,4.56,81874.0,-63286.0,139210.0,-0.63,-43.60,170.03,52.31,136.0,-0.45,0.58,9.59,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.27,96861.0,-31809.0,77111.0,1.45,-24.72,79.61,35.20,219.0,-0.41,0.69,11.17,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.83,48.0,X-SC,0.83,106974.0,4970.0,38960.0,0.65,4.87,36.42,43.06,122.0,0.13,0.76,22.82,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,47.0,X-SC,2.56,108810.0,-19128.0,66265.0,-0.11,-14.95,60.90,36.84,143.0,-0.29,0.78,11.39,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.01,51.0,X-LC,12.73,283316.0,-62640.0,129334.0,1.75,-18.11,45.65,19.28,1.0,-0.48,2.02,4.74,X40,ATH,IT
39,INFY,2275.00,-20.32,44.0,X-LC,7.64,310341.0,-2547.0,174225.0,1.10,-0.81,56.14,54.87,3.0,-0.01,2.22,5.78,X40,BTT,IT
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201835.0,1697.0,19195.0,0.15,0.85,9.51,10.44,4.0,0.09,1.44,6.46,X40,NTT,FMCG
82,VBL,671.64,-18.99,50.0,X-LC,10.52,291598.0,-24244.0,136905.0,-0.97,-7.68,46.95,35.67,5.0,-0.18,2.08,5.11,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,53.0,H-LC,6.89,251544.0,-10075.0,129294.0,0.74,-3.85,51.40,45.57,7.0,-0.08,1.80,7.03,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7027.78,66.0,L-SC,6.47,78235.0,-15575.0,92192.0,3.45,-16.60,117.84,81.67,269.0,-0.17,0.56,51.51,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.65,82272.0,-31277.0,71330.0,1.13,-27.54,86.70,35.27,268.0,-0.44,0.59,101.36,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,46.0,H-SC,9.43,90660.0,-7320.0,28921.0,1.61,-7.47,31.90,22.05,152.0,-0.25,0.65,31.78,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101258.0,9760.0,68997.0,-2.48,10.67,68.14,86.07,223.0,0.14,0.72,47.04,XR,NTT,DURABLES
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124337.0,-960.0,59893.0,3.12,-0.77,48.17,47.03,141.0,-0.02,0.89,55.68,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-188.64,73.0,M-SC,19.76,153999.0,4779.0,139400.0,2.61,3.20,90.52,96.62,208.0,0.03,1.10,66.66,XY25,NTT,FINANCE
77,UJJIVANSFB,60.0,119.37,69.0,H-SC,12.98,138429.0,-4050.0,23574.0,5.23,-2.84,17.03,13.70,163.0,-0.17,0.99,63.38,OX40N,NTT,BANKS
68,SURYODAY,240.0,66.67,62.0,H-SC,11.17,152235.0,-26836.0,92559.0,5.63,-14.99,60.80,36.71,135.0,-0.29,1.09,51.05,XR,NTT,BANKS
4,ANGELONE,3033.0,16.33,70.0,X-SC,8.27,205968.0,14962.0,42738.0,0.61,7.83,20.75,30.21,157.0,0.35,1.47,30.35,X40N,NTT,FINANCE
67,STARHEALTH,761.0,21.02,66.0,H-SC,9.77,269416.0,-11707.0,144569.0,-1.73,-4.16,53.66,47.26,144.0,-0.08,1.92,44.35,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.55
1,25,44.21
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.36
LC    33.87
MC    21.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.56
X40      14.79
XY25     12.43
XR       11.06
X40N     10.23
AR        7.98
OX40N     7.64
X200      1.80
X5K       1.52
SR        1.10
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.51
X-LC    22.90
X-MC    16.01
M-SC    12.71
M-LC     5.23
H-LC     4.71
X-SC     4.65
H-MC     3.85
M-MC     1.60
L-SC     1.49
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.16,-3.49,37.60
IT,12.64,-18.80,82.92
FINANCE,12.21,-10.61,60.74
MISC,6.88,-19.14,85.15
BANKS,6.59,-8.41,65.44
PAINTS,5.68,-16.11,33.41
ELECTRICAL,5.34,-10.19,48.85
INSURANCE,3.94,-0.03,37.43
AUTO,2.80,-46.64,116.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3298272.0,22
XR,1363278.0,14
AR,1212687.0,9
X40,763272.0,10
X40N,593330.0,8
OX40N,577485.0,10
XY25,444376.0,8
SR,266662.0,2
X5K,105554.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3429770.0,25
M-SC,1263422.0,15
X-MC,1085565.0,11
X-LC,1030123.0,13
X-SC,528161.0,6
H-MC,341784.0,3
H-LC,279928.0,3
M-LC,278700.0,4
L-SC,252612.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1174821.0      6
           AR         842479.0      5
           XR         818711.0      7
M-SC       XY24       693026.0      6
X-MC       XY24       590238.0      4
X-LC       X40        472732.0      5
           XY24       306514.0      2
X-SC       X40N       287173.0      4
H-SC       OX40N      267204.0      4
           SR         266662.0      2
X-MC       X40        213429.0      4
H-LC       AR         207196.0      2
X-LC       X40N       187398.0      3
H-MC       XY24       177530.0      1
M-MC       XY24       168626.0      1
X-SC       XY24       163877.0      1
L-SC       XR         163522.0      2
X-MC       XY25       163139.0      2
M-SC       AR         163012.0      2
           OX40N      149319.0      4
           XY25       139400.0      1
X-MC       X40N       118759.0      1
M-SC       XR         118665.0      2
M-LC       XR         111000.0      1
           X5K        105554.0      1
H-MC       XR          92382.0      1
L-SC       OX40N       89090.0      1
X-SC       X40         77111.0      1
H-LC       X200        72732.0      1
H-MC       OX40N       71872.0      1
X-LC       XY25        63479.0      3
H-SC       MH          59893.0      1
L-MC       XR          58998.0      1
L-LC       XY25        39852.0      1
M-LC       XY25        38506.0      1
           XY24        23640.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
